Please remember to update the token. For multiplayer to work, each player needs a different token. Please feel free to use multiple tokens in the same group.

In [1]:
#Update your token
STUDENT_TOKEN = 'Juan Rubiales'

There should be no need to change the code below.

In [2]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'
  
  def get_winner(self):
    return None

  def actions(self):
    return []

  def print(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('Creating new game...')
    id = new_game(game_type, multi_player)

  print('Joining game with id: %s' % id)
  player = join_game(game_type, id)

  print('Playing as %s' % player)

  game = game_state(game_type, id, GameClass)
  print('Waiting for the other player to join...')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('Server says there is a draw')
      else:
        print('Server says your agent ' + ('WON' if game.player == player else 'LOST'))
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      print(move)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

There is almost certainly no need to change this code, but if you must you can.

In [3]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class GinRummy(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return [] # this should return the possible actions

  def get_winner(self):
    return '.' # this should return the actual winner

  def other_player(self):
    if self.player == 'O': return 'X'
    if self.player == 'X': return 'O'

  def letter_to_symbol(letter):
    if letter == 'S': return '\u2664'
    if letter == 'H': return '\u2665'
    if letter == 'D': return '\u2666'
    if letter == 'C': return '\u2667'
    return letter

  def print_game(self):
    if self.player == 'O':
      print('X: %i, \033[1mO: %i\033[0m' % tuple(self.state['score']))
    else:
      print('\033[1mX: %i\033[0m, O: %i' % tuple(self.state['score']))
  
    for card in self.get_hand():
      print('%i%s' % (card[0], GinRummy.letter_to_symbol(card[1])), end = ' ')
    print()

    if self.state['discard']:
      print('Discard: %i%s' % (self.state['discard'][0], GinRummy.letter_to_symbol(self.state['discard'][1]) ))
    else:
      print('Discard: empty')


  def get_hand(self):
    return self.state['hand']

  def get_hand_state(self):
    return self.state['hand_state']

Instead of changing this code for a random solver, write new code that also returns the next move. For example, create a new "amazing_solver".

Solvers need to return lists:
*   If the hand status is 'pick', it needes to be ['deck'] or ['discard'] 
*   If the hand status is 'discard', it needs to be the action (discard, knock, gin), and the value and suit of the card to discard (there is always a discard card). E.g. ['gin', 4, 'H']

In [4]:
def random_solver(game):
  hand_state = game.get_hand_state()

  if hand_state == 'pick':
    return ['deck'] if random.randint(0, 1) == 0 else ['discard']
  else:
    hand = game.get_hand()
    card = hand[random.randint(0, len(hand) - 1)]
    return ['discard', card[0], card[1]]

This code is the one that starts the game. In different games you will need to change the class (GinRummy is the one for gin rummy game).

The 'multi_player' variable can be True or False. If you set multi_palyer to True, you will need to provide a game_id against which to play or if using None a new game will be created (which another player can join).

In [5]:
game_loop(random_solver, GinRummy, 'rummy', multi_player=False, id=None)

Creating new game...
Joining game with id: 30
Playing as X
Waiting for the other player to join...
X: 0, O: 0
10♤ 10♥ 11♤ 8♤ 7♥ 10♦ 2♤ 7♦ 11♧ 12♤ 
Discard: 2♥
Making next move...
['discard']
b'Valid move'
X: 0, O: 0
10♤ 10♥ 11♤ 8♤ 7♥ 10♦ 2♤ 7♦ 11♧ 12♤ 2♥ 
Discard: empty
Making next move...
['discard', 2, 'S']
b'Valid move'
X: 0, O: 0
10♤ 10♥ 11♤ 8♤ 7♥ 10♦ 7♦ 11♧ 12♤ 2♥ 
Discard: 8♥
Making next move...
['discard']
b'Valid move'
X: 0, O: 0
10♤ 10♥ 11♤ 8♤ 7♥ 10♦ 7♦ 11♧ 12♤ 2♥ 8♥ 
Discard: 2♤
Making next move...
['discard', 11, 'S']
b'Valid move'
X: 0, O: 0
10♤ 10♥ 8♤ 7♥ 10♦ 7♦ 11♧ 12♤ 2♥ 8♥ 
Discard: 6♤
Making next move...
['discard']
b'Valid move'
X: 0, O: 0
10♤ 10♥ 8♤ 7♥ 10♦ 7♦ 11♧ 12♤ 2♥ 8♥ 6♤ 
Discard: 11♤
Making next move...
['discard', 2, 'H']
b'Valid move'
X: 0, O: 0
10♤ 10♥ 8♤ 7♥ 10♦ 7♦ 11♧ 12♤ 8♥ 6♤ 
Discard: 8♦
Making next move...
['discard']
b'Valid move'
X: 0, O: 0
10♤ 10♥ 8♤ 7♥ 10♦ 7♦ 11♧ 12♤ 8♥ 6♤ 8♦ 
Discard: 2♥
Making next move...
['discard', 10, 'H']
b'Valid move'
X: 0, O

KeyboardInterrupt: 